# Geographic Mapping and Analysis

This notebook handles geographic mapping and analysis including:
- Geocoding country names to coordinates
- Processing geographic data
- Analyzing host-pathogen relationships by geography
- Determining zoonotic potential

**Author:** [Your Name]  
**Date:** [Date]  
**Version:** 1.0

## Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import time
import os
from pathlib import Path

# Set up paths
project_root = Path().resolve().parent
data_dir = project_root / 'data' / 'geographic_data'
results_dir = project_root / 'results' / 'geographic_analysis'

# Create results directory if it doesn't exist
results_dir.mkdir(parents=True, exist_ok=True)

print(f"Data directory: {data_dir}")
print(f"Results directory: {results_dir}")

## Step 1: Generate Country Coordinates

In [ ]:
# Load the main dataset
metadata_file = data_dir / 'strain_metadata.xlsx'

if metadata_file.exists():
    df_main = pd.read_excel(metadata_file)
    print(f"Loaded metadata for {len(df_main)} strains")
    
    # Create a unique list of countries from the dataframe
    if 'Country' in df_main.columns:
        countries = list(df_main["Country"].dropna().unique())
        print(f"Found {len(countries)} unique countries")
        
        # Convert the list to a DataFrame
        df_countries = pd.DataFrame(countries, columns=['Country'])
        display(df_countries.head())
    else:
        print("Country column not found in metadata")
else:
    print(f"Metadata file not found: {metadata_file}")

## Step 2: Geocode Countries to Coordinates

In [ ]:
# Initialize geolocator
geolocator = Nominatim(user_agent="PasteurellaPhylogenomics")

# Geolocate function with error handling
def geolocate(country):
    """
    Geocode a country name to latitude and longitude coordinates.
    
    Args:
        country (str): Country name to geocode
        
    Returns:
        tuple: (latitude, longitude) or np.nan if not found
    """
    try:
        # Introduce sleep to avoid hitting rate limit
        time.sleep(1)
        location = geolocator.geocode(country)
        if location is not None:
            return (location.latitude, location.longitude)
        else:
            print(f'{country} not recognized')
            return np.nan
    except Exception as e:
        print(f'Error geocoding {country}: {e}')
        return np.nan

# Apply geocoding to countries
if 'df_countries' in locals():
    print("Starting geocoding process...")
    df_countries['Coordinates'] = df_countries['Country'].apply(geolocate)
    
    # Separate latitude and longitude
    df_countries['Latitude'] = df_countries['Coordinates'].apply(lambda x: x[0] if pd.notna(x) else np.nan)
    df_countries['Longitude'] = df_countries['Coordinates'].apply(lambda x: x[1] if pd.notna(x) else np.nan)
    
    # Save country coordinates
    coords_file = data_dir / 'country_coordinates.xlsx'
    df_countries.to_excel(coords_file, index=False)
    print(f"Country coordinates saved to: {coords_file}")
    
    # Display results
    print(f"Successfully geocoded {df_countries['Latitude'].notna().sum()} out of {len(df_countries)} countries")
    display(df_countries.head())

## Step 3: Handle Missing Countries

In [ ]:
# Check for missing coordinates and handle manually if needed
if 'df_countries' in locals():
    missing_coords = df_countries[df_countries['Latitude'].isna()]
    
    if len(missing_coords) > 0:
        print(f"Countries with missing coordinates: {len(missing_coords)}")
        print(missing_coords['Country'].tolist())
        
        # Manual coordinates for problematic countries
        manual_coords = {
            'Czechoslovakia': (49.75, 15.5),  # Historical country, use Czech Republic coordinates
            'USSR': (55.0, 37.0),  # Historical country, use Russia coordinates
            # Add more manual coordinates as needed
        }
        
        # Apply manual coordinates
        for country, coords in manual_coords.items():
            if country in missing_coords['Country'].values:
                idx = df_countries[df_countries['Country'] == country].index[0]
                df_countries.loc[idx, 'Latitude'] = coords[0]
                df_countries.loc[idx, 'Longitude'] = coords[1]
                df_countries.loc[idx, 'Coordinates'] = coords
                print(f"Added manual coordinates for {country}: {coords}")
        
        # Save updated coordinates
        coords_file = data_dir / 'country_coordinates_complete.xlsx'
        df_countries.to_excel(coords_file, index=False)
        print(f"Updated coordinates saved to: {coords_file}")

## Step 4: Map Coordinates to Original Dataset

In [ ]:
# Create coordinate mapping dictionary
if 'df_countries' in locals() and 'df_main' in locals():
    # Create mapping dictionary
    country_coord_dict = dict(zip(df_countries['Country'], df_countries['Coordinates']))
    country_lat_dict = dict(zip(df_countries['Country'], df_countries['Latitude']))
    country_lon_dict = dict(zip(df_countries['Country'], df_countries['Longitude']))
    
    # Map coordinates to main dataset
    df_main['Coordinates'] = df_main['Country'].map(country_coord_dict)
    df_main['Latitude'] = df_main['Country'].map(country_lat_dict)
    df_main['Longitude'] = df_main['Country'].map(country_lon_dict)
    
    # Save enhanced dataset
    enhanced_file = data_dir / 'strain_metadata_with_coordinates.xlsx'
    df_main.to_excel(enhanced_file, index=False)
    print(f"Enhanced dataset with coordinates saved to: {enhanced_file}")
    
    # Check mapping success
    mapped_count = df_main['Latitude'].notna().sum()
    print(f"Successfully mapped coordinates for {mapped_count} out of {len(df_main)} strains")

## Step 5: Host Affinity Analysis

In [ ]:
# Analyze host affinity and determine zoonotic potential
if 'df_main' in locals():
    # Convert Host and Species columns to string type
    df_analysis = df_main.copy()
    df_analysis['Host'] = df_analysis['Host'].astype(str)
    df_analysis['Species'] = df_analysis['Species'].astype(str)
    
    # Define function to determine affinity
    def determine_affinity(row):
        """
        Determine host affinity based on host information.
        
        Args:
            row: DataFrame row containing Host information
            
        Returns:
            str: 'Human', 'Animals', 'Zoonotic', or None
        """
        host_info = str(row['Host']).lower()
        
        if 'homo sapiens' in host_info and 'animal' in host_info:
            return 'Zoonotic'
        elif 'homo sapiens' in host_info or 'human' in host_info:
            return 'Human'
        elif 'animal' in host_info or any(animal in host_info for animal in ['pig', 'cattle', 'chicken', 'dog', 'cat']):
            return 'Animals'
        else:
            return 'Unknown'
    
    # Apply affinity determination
    df_analysis['Affinity'] = df_analysis.apply(determine_affinity, axis=1)
    
    # Display affinity distribution
    print("Host affinity distribution:")
    affinity_counts = df_analysis['Affinity'].value_counts()
    print(affinity_counts)
    
    # Save analysis results
    affinity_file = results_dir / 'host_affinity_analysis.xlsx'
    df_analysis.to_excel(affinity_file, index=False)
    print(f"Host affinity analysis saved to: {affinity_file}")

## Step 6: Geographic Distribution Summary

In [ ]:
# Generate geographic distribution summary
if 'df_analysis' in locals():
    # Group by species, country, and affinity
    geo_summary = df_analysis.groupby(['Species', 'Country', 'Affinity']).size().reset_index(name='Count')
    
    # Identify zoonotic species (present in both human and animal hosts)
    species_affinity = df_analysis.groupby('Species')['Affinity'].unique().reset_index()
    zoonotic_species = []
    
    for _, row in species_affinity.iterrows():
        affinities = row['Affinity']
        if len(affinities) > 1 and ('Human' in affinities and 'Animals' in affinities):
            zoonotic_species.append(row['Species'])
    
    print(f"Identified {len(zoonotic_species)} potentially zoonotic species:")
    for species in zoonotic_species:
        print(f"  - {species}")
    
    # Update affinity for zoonotic species
    geo_summary.loc[geo_summary['Species'].isin(zoonotic_species), 'Affinity'] = 'Zoonotic'
    
    # Save geographic summary
    geo_file = results_dir / 'geographic_distribution_summary.xlsx'
    geo_summary.to_excel(geo_file, index=False)
    print(f"Geographic distribution summary saved to: {geo_file}")
    
    # Display summary statistics
    print("\nGeographic distribution summary:")
    print(f"Total species: {geo_summary['Species'].nunique()}")
    print(f"Total countries: {geo_summary['Country'].nunique()}")
    print(f"Zoonotic species: {len(zoonotic_species)}")
    
    display(geo_summary.head(10))

## Summary and Next Steps

This notebook has completed:
1. ✅ Geocoding of country names to coordinates
2. ✅ Processing of geographic data
3. ✅ Host affinity analysis
4. ✅ Identification of zoonotic potential
5. ✅ Geographic distribution summary

**Output files generated:**
- `country_coordinates.xlsx`: Country coordinate mapping
- `strain_metadata_with_coordinates.xlsx`: Enhanced metadata with coordinates
- `host_affinity_analysis.xlsx`: Host affinity analysis results
- `geographic_distribution_summary.xlsx`: Geographic distribution summary

**Next steps:**
- Use `04_Visualization_and_Plots.ipynb` to create geographic visualizations
- Proceed to `05_Statistical_Analysis.ipynb` for statistical analysis